<a href="https://colab.research.google.com/github/Jesimsim/ds-section2-project/blob/main/section2_project_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **아파트 실거래가 예측**

In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import math


# from sklearn. import
# from sklearn.metric import f1_score

#### **디스플레이 옵션**

In [ ]:
# 디스플레이 옵션 조정
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option("max_colwidth", 2000)

# 숫자 전체 잘보이게
pd.set_option('float_format', '{:}'.format)

# 숫자 소수점한자리만 보이게
pd.set_option('display.float_format', '{:,.1f}'.format)

In [ ]:
pd.reset_option("^display")

#### **필요코드 설치**

In [1]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
import sys

if "google.colab" in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install --upgrade xgboost


import xgboost
import warnings

warnings.filterwarnings("ignore")
# xgboost.config.set_config(verbosity=0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 174 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.1 MB/s 
     |████████████████████████████████| 133 kB 36.2 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=708f4953f075b5d1aef2437fa7f4e30eb55be740f5694e1d61e32db032fe3f48
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 193.6 MB 70 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
#한글폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (2,891 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

## **아파트 데이터**

#### **데이터 병합하기 - 아파트**

In [ ]:
# 데이터 불러오기 및 병합

# 데이터 확인
import os
os.listdir('/content/AptALL')

FileNotFoundError: ignored

In [ ]:
forders= os.listdir('/content/AptALL')
%cd /content/AptALL

/content/AptALL


In [ ]:
# 데이터 병합 - 월별
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949', encoding_errors='ignore',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df.to_csv("Apt_2022_all.csv", encoding='utf-8')

In [ ]:
# 병합오류 확인
pd.read_csv('Apt_2017_all.csv', encoding='utf-8', encoding_errors='ignore') #2022(다시병합), 2017(언어 오류, 병합오류). 2013(병합오류)

In [ ]:
#오류데이터프레임 수정 > 업로드 제대로 마칠때까지 기다리기
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df = df.drop(['Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18','Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'], axis=1)
df.columns
df.to_csv("Apt_2013_all.csv", encoding='utf-8')

In [ ]:
# 데이터 병합 - 총

import os

forders= os.listdir('/content/AptALL')
%cd /content/AptALL

df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore', index_col=0)
    df = pd.concat([df, df_1], axis=0)


df.to_csv("Apt_total.csv", encoding='utf-8')

/content/AptALL


IsADirectoryError: ignored

In [ ]:
# 에러파일 확인
forders= os.listdir('/content/AptALL')
%cd /content/AptALL
pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore') #2013 unnamed:칼럼 2개


/content/AptALL


,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,0,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.43,202102,9,"145,000",6,1984,개포로 307,20210223.0,-,-
1,1,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,53.06,202102,1,"200,000",12,1983,개포로 516,NaN,-,-
2,2,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,73.02,202102,28,"240,500",7,1983,개포로 516,NaN,-,-
3,3,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,182.20,202102,5,"327,000",11,1984,언주로 110,NaN,-,-
4,4,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,186.16,202102,6,"332,000",14,1984,언주로 110,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43413,1438,서울특별시 중랑구 신내동,650,650.0,0.0,신내6대주,59.76,202111,30,"75,000",12,1996,신내로19길 42,NaN,중개거래,서울 중랑구
43414,1439,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,59.49,202111,5,"80,000",20,2014,신내역로1길 85,NaN,중개거래,"서울 동대문구, 서울 중랑구"
43415,1440,서울특별시 중랑구 신내동,654,654.0,0.0,진로아파트,101.86,202111,4,"100,000",7,1995,신내로17길 41,NaN,중개거래,서울 중랑구
43416,1441,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산아파트,84.99,202111,22,"86,000",14,1995,신내로 155,NaN,중개거래,서울 중랑구


In [ ]:
# 에러파일 수정
df = pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore')
df = df.drop('Unnamed: 0.1', axis=1)
df.to_csv("Apt_2013_all(2).csv", encoding='utf-8')

In [ ]:
# 에러파일 확인 2 / 언어오류
df = pd.DataFrame()
forder = [
  'Apt_2019_all.csv',
 'Apt_2013_all.csv',
 'Apt_2017_all.csv',
 'Apt_2010_all.csv',
 'Apt_2011_all.csv',
 'Apt_2022_all.csv',
 'Apt_2015_all.csv',
 'Apt_2012_all.csv',
 'Apt_2020_all.csv',
 'Apt_2018_all.csv',
 'Apt_2021_all.csv',
 'Apt_2014_all.csv',
 'Apt_2016_all.csv']


for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore')
    df = pd.concat([df, df_1], axis=0)

df.head()
# df.to_csv("Apt_check1.csv", encoding='utf-8')

,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1
0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN


In [ ]:
# 오류 확인
pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore')

,index,Unnamed: 0,Unnamed: 0.1,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1.1
0,0,0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924455,924455,110317,10477,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,21,"30,000",7.0,1997.0,동일로 752,NaN,-,-,NaN
924456,924456,110318,10478,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,201604,21,"39,000",15.0,1997.0,동일로 752,NaN,-,-,NaN
924457,924457,110319,10479,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,25,"32,000",21.0,1997.0,동일로 752,NaN,-,-,NaN
924458,924458,110320,10480,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),50.37,201604,26,"27,200",6.0,1997.0,동일로 752,NaN,-,-,NaN


- 파일 가져올때 https://hyunsitstory.tistory.com/entry/python-Jupyter-폴더-내-여러-개의-csv파일을-하나의-데이터-프레임으로-한-번에-합치기
- https://boleumdal.tistory.com/entry/python-폴더-안에-있는-파일-한번에-불러오기 
- 액셀 열때, 텍스트가져오기 마법사 https://support.microsoft.com/ko-kr/office/%ED%85%8D%EC%8A%A4%ED%8A%B8-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0-%EB%A7%88%EB%B2%95%EC%82%AC-c5b02af6-fda1-4440-899f-f78bafe41857


#### **데이터 줄이기 - 아파트**
- 연도2013-22, 컬럼명변경

In [ ]:
df = pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore', engine='python')
df = pd.DataFrame(df, columns=['시군구','번지','본번','부번','단지명','전용면적(㎡)','계약년월'
    ,'계약일','거래금액(만원)','층','건축년도','도로명','해제사유발생일','거래유형','중개사소재지'])
df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',','')

df = df.rename(columns={'시군구': 'address', '번지': 'address_num', '본번' : 'address_num_first', '부번':'address_num_second',
                   '단지명' : 'apt_name', '전용면적(㎡)' : 'area','계약년월':'date',
                   '계약일':'date_day','거래금액(만원)':'price','층':'floor','건축년도':'construction_year','도로명':'address_street',
                   '해제사유발생일':'date_on_sale','거래유형':'deal_type','중개사소재지':'real_estate_office'})

df = df[df['date'] >= 201301]

,address,address_num,address_num_first,address_num_second,apt_name,area,date,date_day,price,floor,construction_year,address_street,date_on_sale,deal_type,real_estate_office
0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,202000,5.0,2004.0,개포로109길 69,NaN,-,-
1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,218000,9.0,2004.0,개포로109길 69,NaN,-,-
2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,204000,5.0,1982.0,개포로 310,NaN,-,-
3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,220000,2.0,1982.0,개포로 310,NaN,-,-
4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,205000,1.0,1982.0,개포로 310,NaN,-,-


In [ ]:
df = df.sort_values('date')
df.date.unique()

array([201301, 201302, 201303, 201304, 201305, 201306, 201307, 201308,
       201309, 201310, 201311, 201312, 201401, 201402, 201403, 201404,
       201405, 201406, 201407, 201408, 201409, 201410, 201411, 201412,
       201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508,
       201509, 201510, 201511, 201512, 201601, 201602, 201603, 201604,
       201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612,
       201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108,
       202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204,
      

In [ ]:
df.to_csv("Apt_2013_22.csv", encoding='utf-8')

#### **데이터 불러오기 - 드라이브**

In [4]:
# 드라이브 불러오기
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [46]:
# 데이터불러오기 / 아파트
filename = '/content/drive/MyDrive/Apt_2013_22.csv'
df = pd.read_csv(filename, encoding='utf-8', encoding_errors='ignore', engine='python', index_col=0) #15~30초

In [5]:
# URL 불러오기 / 잘작동안되는듯
url = 'https://drive.google.com/file/d/1mc9hbRuTSzlMwVr_ZcXNmvLZKFHmnnKF/view?usp=sharing'
df = pd.read_csv(url, encoding='utf-8', encoding_errors='ignore', engine='python', index_col=0)

ParserError: ignored

In [7]:
#기준금리 불러오기

filename2 = '한국은행_기준금리.csv'
df_baserate = pd.read_csv(filename2, encoding='utf-8', encoding_errors='ignore', engine='python', names=['date', 'baserate'], header=None)

df_baserate['date_year'] = df_baserate.date.str.split('-').str[0]
df_baserate['date_month'] = df_baserate.date.str.split('-').str[1]
df_baserate['date_day'] = df_baserate.date.str.split('-').str[2]
df_baserate['date'] = df_baserate.date.str.replace('-', '')

In [8]:
# 브랜드 불러오기
filename3 = 'brand.csv'
df_brand = pd.read_csv(filename3, encoding='utf-8', encoding_errors='ignore', engine='python')
df_brand.head()
brand_highend = list(df_brand[:4]['브랜드'])
df_brand = df_brand.iloc[4:,:]

df_brand_sort = df_brand.groupby('브랜드').count()
brand_top = list(df_brand_sort[df_brand_sort.연도 >= 10].index)
brand_good = list(df_brand_sort[df_brand_sort.연도 < 10].index)
brand_highend.extend(['센트라스','타워팰리스'])
brand_top.extend(['트라팰리스','리체','텐즈힐','레이크팰리스'])
brand_good.extend(['View', 'VIEW','리더스뷰','그레이스힐','SK','선경아파트'])
brand_highend, brand_top, brand_good  #호반써밋 제외

(['써밋', '디에이치', '르엘', '오티에르', '센트라스', '타워팰리스'],
 ['e편한세상',
  '더샵',
  '래미안',
  '롯데캐슬',
  '아이파크',
  '자이',
  '푸르지오',
  '힐스테이트',
  '트라팰리스',
  '리체',
  '텐즈힐',
  '레이크팰리스'],
 ['view',
  '꿈에그린',
  '린',
  '베르디움',
  '위브',
  '포레나',
  '휴먼시아',
  'View',
  'VIEW',
  '리더스뷰',
  '그레이스힐',
  'SK',
  '선경아파트'])

In [9]:
# 소비자물가지수 불러오기

filename4 = '소비자물가지수_1010_2209.csv'
df_cpi = pd.read_csv(filename4, encoding='utf-8', encoding_errors='ignore', engine='python', names=['연월','소비자물가지수'], header=None).iloc[1:]
df_cpi['연월'] = df_cpi['연월'].astype('str')
df_cpi['year'] = df_cpi['연월'].str.split('\.', -1).str[0]
df_cpi['month'] = df_cpi['연월'].str.split('\.', -1).str[1]
df_cpi['연월'] = df_cpi['연월'].str.replace('\.','')

#오류수정
df_cpi.month.unique() # df_cpi.month.unique() 1 > 01로
for i in range(0,df_cpi.shape[0]):
  if df_cpi.iloc[i, -1] == '1':
    df_cpi.iloc[i, -1] = '01'
df_cpi.month.unique()

df_cpi = df_cpi[df_cpi['year'] >= '2013'].reset_index(drop=True)
df_cpi.head()

,연월,소비자물가지수,year,month
0,201301,92.728,2013,01
1,201302,93.038,2013,02
2,201303,92.952,2013,03
3,201304,92.823,2013,04
4,201305,92.823,2013,05


In [ ]:
df_park

,add_gu,count,duplicate,area,year
1,소계,"2,868",33,"168,896.8",2019
2,종로구,106,2,"11,556.1",2019
3,중구,73,2,"3,155.1",2019
4,용산구,110,1,"1,779.6",2019
5,성동구,86,1,"3,100.0",2019
...,...,...,...,...,...
256,관악구,113,3,"12,431.4",2020
257,서초구,181,1,"15,072.6",2020
258,강남구,162,1,"7,768.7",2020
259,송파구,173,1,"4,771.4",2020


In [ ]:
df_area

,자치구,area_m2
0,종로구,23910000.0
1,중구,9960000.0
2,용산구,21870000.0
3,성동구,16860000.0
4,광진구,17060000.0
5,동대문구,14220000.0
6,중랑구,18500000.0
7,성북구,24580000.0
8,강북구,23600000.0
9,도봉구,20650000.0


In [10]:
#공원 불러오기 / area 1000m2 == 0.001km2
filename5 = 'park.csv'
df_park = pd.read_csv(filename5, encoding='utf-8', encoding_errors='ignore', engine='python', names=['add_gu','count','duplicate','area','year'], header=None).iloc[1:,:]

filename6 = '자치구별면적.csv'
df_area = pd.read_csv(filename6,encoding='utf-8', encoding_errors='ignore', engine='python').iloc[:25,:]
df_area['면적_km2'] = df_area['면적_km2'].astype('float') * 1000000
df_area = df_area.rename(columns={'면적_km2':'area_m2'})
df_area

# 해당하는 값에 할당 / 면적 m2으로 환산
lst_park = df_area.자치구.values
for i in range(0, len(lst_park)): #30초!!!
  result = df_park[df_park['add_gu'].str.contains(lst_park[i])]
  df_park.loc[list(result.index), 'area_gu'] = df_area.iloc[i, 1]
df_park['area'] = df_park['area'].str.replace(',','').astype('float') * 1000 
df_park['ratio_park'] = df_park.area / df_park.area_gu
df_park

,add_gu,count,duplicate,area,year,area_gu,ratio_park
1,소계,"2,868",33,168896800.0,2019,NaN,NaN
2,종로구,106,2,11556100.0,2019,23910000.0,0.483317
3,중구,73,2,3155100.0,2019,9960000.0,0.316777
4,용산구,110,1,1779600.0,2019,21870000.0,0.081372
5,성동구,86,1,3100000.0,2019,16860000.0,0.183867
...,...,...,...,...,...,...,...
256,관악구,113,3,12431400.0,2020,29570000.0,0.420406
257,서초구,181,1,15072600.0,2020,46980000.0,0.320830
258,강남구,162,1,7768700.0,2020,39500000.0,0.196676
259,송파구,173,1,4771400.0,2020,33870000.0,0.140874


In [54]:
filename7 = '자치구별인구수.csv'
df_people = pd.read_csv(filename7, encoding='cp949', encoding_errors='ignore', engine='python').iloc[2:].reset_index(drop=True)
df_people = df_people.rename(columns={'행정구역(시군구)별':'add_gu'})
df_people


,add_gu,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,종로구,160070,156993,154986,152737,154770,153065,151290,149384,144683
1,중구,130465,128065,125733,125249,125709,125725,126171,125240,122499
2,용산구,239740,235951,233342,230241,229161,228999,228670,230040,222953
3,성동구,299337,296086,297003,299259,304808,308221,300889,293556,285990
4,광진구,368927,363354,360369,357215,357703,355559,351350,346682,339996
5,동대문구,364273,363687,360153,355069,350647,348052,346194,342837,337400
6,중랑구,416798,418836,413909,411005,408226,403209,397015,394702,387350
7,성북구,476589,466706,459275,450355,444055,435868,442650,437153,430528
8,강북구,338707,335025,330873,327195,324479,319164,313954,308055,299182
9,도봉구,358582,353709,351242,348220,344166,339413,333362,325257,317366


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784225 entries, 141040 to 353823
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   address             784225 non-null  object 
 1   address_num         784004 non-null  object 
 2   address_num_first   784150 non-null  float64
 3   address_num_second  784150 non-null  float64
 4   apt_name            784225 non-null  object 
 5   area                784225 non-null  float64
 6   date                784225 non-null  int64  
 7   date_day            784225 non-null  int64  
 8   price               784225 non-null  int64  
 9   floor               784224 non-null  float64
 10  construction_year   784223 non-null  float64
 11  address_street      784225 non-null  object 
 12  date_on_sale        5091 non-null    float64
 13  deal_type           784223 non-null  object 
 14  real_estate_office  784223 non-null  object 
 15  area_type           784225 no

In [11]:
df.head()

,address,address_num,address_num_first,address_num_second,apt_name,area,date,date_day,price,floor,construction_year,address_street,date_on_sale,deal_type,real_estate_office
141040,서울특별시 동대문구 휘경동,109,109.0,0.0,롯데낙천대,112.030,201301,25,38000,2.0,2003.0,외대역동로1길 24,NaN,-,-
141578,서울특별시 성동구 금호동4가,235,235.0,0.0,서울숲푸르지오2차,114.771,201301,8,75352,1.0,2012.0,금호로 17,NaN,-,-
141579,서울특별시 성동구 금호동4가,235,235.0,0.0,서울숲푸르지오2차,114.771,201301,31,83712,4.0,2012.0,금호로 17,NaN,-,-
141580,서울특별시 성동구 도선동,69,69.0,0.0,삼성쉐르빌,84.740,201301,15,40000,12.0,2006.0,무학로6길 50,NaN,-,-
141581,서울특별시 성동구 마장동,819,819.0,0.0,마장동중앙하이츠,84.800,201301,22,32850,2.0,1999.0,마장로 262,NaN,-,-


In [ ]:
df_brand

,연도,브랜드,건설사,브랜드2
4,2022.0,자이,GS건설,NaN
5,2022.0,푸르지오,대우건설,NaN
6,2022.0,래미안,삼성물산,NaN
7,2022.0,힐스테이트,현대건설,NaN
8,2022.0,롯데캐슬,롯데건설,NaN
...,...,...,...,...
99,2013.0,아이파크,NaN,NaN
100,2013.0,롯데캐슬,NaN,NaN
101,2013.0,더샵,NaN,NaN
102,2013.0,위브,NaN,NaN


##### **실험용**

In [ ]:
# 실험용
df2 = df.copy()
df2['add_gu'] = df.address.str.split(' ', -1).str[1]
df2['add_dong'] = df.address.str.split(' ', -1).str[2]

In [ ]:
df.address.str.split(' ', -1).str[1]

KeyboardInterrupt: ignored

In [ ]:
df.astype({'date' : 'str'}) 

In [ ]:
df.date.dtypes

In [ ]:
df.date.str[:2]

In [ ]:
# 실험용
df2 = df.copy()
df2 = df.astype({'date' : 'str'}) 
df2['date_year'] = df.date.str[:4]
df2['date_month'] = df.date.astype(object).str[5:]

In [ ]:
df2.head()

In [ ]:
df.date[0] #20191211

In [ ]:
df_baserate.date[0] #20221124

In [ ]:
df.date[0] < df_baserate.date[0] #True

In [ ]:
# 기준금리 실험
# df['baserate'] = ''
# for m in range(0, df_baserate.shape[0]):
#   for i in range(0, df.shape[0]):
#     if df.iloc[i, 0] >= df_baserate.iloc[m, 4]:
#       df.iloc[i, -1] = df_baserate.iloc[m,0]
# 2시간반????이라.... 곤란한데...?
# 데이터 78만개... 음...

##### **데이터 전처리**

In [53]:
# 소비자물가
lst_year = df_cpi.연월.values

df = df.astype({'date' : 'str'})  #date열만 문자열로
# 데이터프레임에서 값 찾아서 할당하기
for i in range(0, len(lst_year)): #30~60초!!!
  result = df[df['date'].str.contains(lst_year[i])]
  df.loc[list(result.date.index), 'cpi'] = df_cpi.iloc[i, 1]

# 계약년월(날짜) 분리
df['date_year'] = df.date.str[:4]
df['date_month'] = df.date.str[4:]

df.head()

,address,address_num,address_num_first,address_num_second,apt_name,area,date,date_day,price,floor,construction_year,address_street,date_on_sale,deal_type,real_estate_office,cpi,date_year,date_month
141040,서울특별시 동대문구 휘경동,109,109.0,0.0,롯데낙천대,112.030,201301,25,38000,2.0,2003.0,외대역동로1길 24,NaN,-,-,93.134,2013,01
141578,서울특별시 성동구 금호동4가,235,235.0,0.0,서울숲푸르지오2차,114.771,201301,8,75352,1.0,2012.0,금호로 17,NaN,-,-,93.134,2013,01
141579,서울특별시 성동구 금호동4가,235,235.0,0.0,서울숲푸르지오2차,114.771,201301,31,83712,4.0,2012.0,금호로 17,NaN,-,-,93.134,2013,01
141580,서울특별시 성동구 도선동,69,69.0,0.0,삼성쉐르빌,84.740,201301,15,40000,12.0,2006.0,무학로6길 50,NaN,-,-,93.134,2013,01
141581,서울특별시 성동구 마장동,819,819.0,0.0,마장동중앙하이츠,84.800,201301,22,32850,2.0,1999.0,마장로 262,NaN,-,-,93.134,2013,01


In [56]:

lst_people = df_people.add_gu.values
lst_people_year = list(df_people.columns)[1:]

year13 = df[df.date_year == '2018']
year13.head()
# 이 부분은 내일...
# for i in range(0, len(lst_park)): #30초!!!
#   result = df_park[df_park['add_gu'].str.contains(lst_people[i])]
# for i in range(0, len(lst_people_year))
#   result = result[result['year']].str.contains(lst_people_year[i]]
#   df_park.loc[list(result.index), 'area_gu'] = df_area.iloc[i, 1]

,address,address_num,address_num_first,address_num_second,apt_name,area,date,date_day,price,floor,construction_year,address_street,date_on_sale,deal_type,real_estate_office,cpi,date_year,date_month
684502,서울특별시 중구 신당동,844,844.0,0.0,남산타운,84.88,201801,23,74000,8.0,2002.0,다산로 32,NaN,-,-,100.041,2018,01
684491,서울특별시 중구 신당동,844,844.0,0.0,남산타운,114.88,201801,19,84800,10.0,2002.0,다산로 32,NaN,-,-,100.041,2018,01
684506,서울특별시 중구 신당동,844,844.0,0.0,남산타운,114.88,201801,24,88000,18.0,2002.0,다산로 32,NaN,-,-,100.041,2018,01
684492,서울특별시 중구 신당동,844,844.0,0.0,남산타운,59.94,201801,19,57400,3.0,2002.0,다산로 32,NaN,-,-,100.041,2018,01
684493,서울특별시 중구 신당동,844,844.0,0.0,남산타운,84.88,201801,20,68750,15.0,2002.0,다산로 32,NaN,-,-,100.041,2018,01


In [57]:
# 주소분리
df['add_gu'] = df.address.str.split(' ', -1).str[1]
df['add_dong'] = df.address.str.split(' ', -1).str[2]

In [58]:
 # 열순서 최종변경
df = df[['date','address','date_year', 'date_month', 'date_day','add_gu','add_dong','address_num','address_num_first','address_num_second','address_street','apt_name','area','floor','construction_year','price','date_on_sale','deal_type','real_estate_office', 'cpi']]

In [59]:
#날짜 / 연월 + 일 합치기
df.date = df.date + df.date_day.astype('str').str.pad(width=2, side='left',fillchar='0')

In [60]:
df.head()

,date,address,date_year,date_month,date_day,add_gu,add_dong,address_num,address_num_first,address_num_second,address_street,apt_name,area,floor,construction_year,price,date_on_sale,deal_type,real_estate_office,cpi
141040,20130125,서울특별시 동대문구 휘경동,2013,01,25,동대문구,휘경동,109,109.0,0.0,외대역동로1길 24,롯데낙천대,112.030,2.0,2003.0,38000,NaN,-,-,93.134
141578,20130108,서울특별시 성동구 금호동4가,2013,01,8,성동구,금호동4가,235,235.0,0.0,금호로 17,서울숲푸르지오2차,114.771,1.0,2012.0,75352,NaN,-,-,93.134
141579,20130131,서울특별시 성동구 금호동4가,2013,01,31,성동구,금호동4가,235,235.0,0.0,금호로 17,서울숲푸르지오2차,114.771,4.0,2012.0,83712,NaN,-,-,93.134
141580,20130115,서울특별시 성동구 도선동,2013,01,15,성동구,도선동,69,69.0,0.0,무학로6길 50,삼성쉐르빌,84.740,12.0,2006.0,40000,NaN,-,-,93.134
141581,20130122,서울특별시 성동구 마장동,2013,01,22,성동구,마장동,819,819.0,0.0,마장로 262,마장동중앙하이츠,84.800,2.0,1999.0,32850,NaN,-,-,93.134


In [61]:
#면적 규모별 타입 분류

def classi_area(x):
  if x >= 135:
    return 5 #대형
  elif x >= 85 :
    return 4 #중대형
  elif x>=65 :
    return 3 #중소형
  elif x>=33 :
    return 2 #소형
  elif x>= 0:
    return 1 #초소형
  else :
    0 #결측치

df['area_type'] = df.area.apply(classi_area)

In [62]:
df.columns

Index(['date', 'address', 'date_year', 'date_month', 'date_day', 'add_gu',
       'add_dong', 'address_num', 'address_num_first', 'address_num_second',
       'address_street', 'apt_name', 'area', 'floor', 'construction_year',
       'price', 'date_on_sale', 'deal_type', 'real_estate_office', 'cpi',
       'area_type'],
      dtype='object')

In [ ]:
# 날짜변경 / df_baserate > 기준금리 데이터프레임

# np.delete(df_baserate['date_start'].values, 27)
# len(np.insert(np.delete(df_baserate['date_start'].values, 27), 0, 'now'))


df_baserate['date_start'] = df_baserate['date']
df_baserate['date_fin'] = np.insert(np.delete(df_baserate['date_start'].values, 27), 0, '20221131').astype('int') -1
df_baserate['date_fin'] = df_baserate['date_fin'].astype('str')
df_baserate = df_baserate[['date_start','date_fin','baserate', 'date_year','date_month','date_day']]
df_baserate = df_baserate.drop(columns='date')

# df_baserate.to_csv('baserate.csv', encoding='utf-8')

KeyError: ignored

In [63]:
# 순서 재정렬
df = df.sort_values('date').reset_index(drop=True)
df.head()

,date,address,date_year,date_month,date_day,add_gu,add_dong,address_num,address_num_first,address_num_second,...,apt_name,area,floor,construction_year,price,date_on_sale,deal_type,real_estate_office,cpi,area_type
0,20130101,서울특별시 은평구 대조동,2013,01,1,은평구,대조동,04월 18일,4.0,18.0,...,아이안빌,31.32,9.0,2012.0,16900,NaN,-,-,93.134,1
1,20130101,서울특별시 구로구 구로동,2013,01,1,구로구,구로동,1280,1280.0,0.0,...,한신휴플러스(1280-0),58.03,12.0,2007.0,34500,NaN,-,-,93.134,2
2,20130101,서울특별시 동작구 사당동,2013,01,1,동작구,사당동,1155,1155.0,0.0,...,대성유니드,84.96,7.0,2009.0,40250,NaN,-,-,93.134,3
3,20130101,서울특별시 은평구 응암동,2013,01,1,은평구,응암동,176,176.0,0.0,...,대우푸르지오,59.58,9.0,2008.0,27500,NaN,-,-,93.134,2
4,20130101,서울특별시 중구 신당동,2013,01,1,중구,신당동,851,851.0,0.0,...,래미안하이베르,117.00,14.0,2011.0,62000,NaN,-,-,93.134,4


In [64]:
# 건물연식
df['building_year'] = df.date_year.astype('int') - df.construction_year



# 리버뷰
riverlist = ['방화동', '가양동', '염창동', '양평동', 
              '당산동', '여의동', '노량진동', '흑석동', '반포동', 
              '잠원동', '신사동', '압구정동', '청담동', '잠실동', 
              '삼성동', '풍남동', '천호동', '암사동', '고덕동', 
              '광장동', '구의동', '자양동', '성수동2가', '성수동1가', 
              '금호동3가', '한남동', '보광동', '서빙고동', '한강로동', 
              '이촌동', '원효로', '도화동', '용강동', '신수동', '서강동', 
              '합정동', '망원동', '상암동']

df['river'] = df.add_dong.map(lambda x : 1 if x in riverlist else 0)



# 브랜드분류 / brand_highend(3), brand_top(2), brand_good(1)  #호반써밋 제외
# 하이엔드

search = '|'.join(brand_highend)
result = df[df['apt_name'].str.contains(search)]
# result.apt_name.unique()
result = result[~result['apt_name'].str.contains('호반써밋')]
df.loc[list(result.apt_name.index), 'apt_brand'] = 3

search = '|'.join(brand_top)
result = df[df['apt_name'].str.contains(search)]
df.loc[list(result.apt_name.index), 'apt_brand'] = 2

search = '|'.join(brand_good)
result = df[df['apt_name'].str.contains(search)]
df.loc[list(result.apt_name.index), 'apt_brand'] = 1

df.apt_brand = df.apt_brand.fillna(0)
df.head()

,date,address,date_year,date_month,date_day,add_gu,add_dong,address_num,address_num_first,address_num_second,...,construction_year,price,date_on_sale,deal_type,real_estate_office,cpi,area_type,building_year,river,apt_brand
0,20130101,서울특별시 은평구 대조동,2013,01,1,은평구,대조동,04월 18일,4.0,18.0,...,2012.0,16900,NaN,-,-,93.134,1,1.0,0,0.0
1,20130101,서울특별시 구로구 구로동,2013,01,1,구로구,구로동,1280,1280.0,0.0,...,2007.0,34500,NaN,-,-,93.134,2,6.0,0,0.0
2,20130101,서울특별시 동작구 사당동,2013,01,1,동작구,사당동,1155,1155.0,0.0,...,2009.0,40250,NaN,-,-,93.134,3,4.0,0,0.0
3,20130101,서울특별시 은평구 응암동,2013,01,1,은평구,응암동,176,176.0,0.0,...,2008.0,27500,NaN,-,-,93.134,2,5.0,0,2.0
4,20130101,서울특별시 중구 신당동,2013,01,1,중구,신당동,851,851.0,0.0,...,2011.0,62000,NaN,-,-,93.134,4,2.0,0,2.0


In [65]:
# 기준금리 분류

def classi_rate(x):
  if (x >= '20221012') & (x <= '20221123'):
    return 3
  elif (x >= '20220825') & (x <= '20221011'):
    return 2.5
  elif (x >= '20220713') & (x <= '20220824'):
    return 2.25
  elif (x >= '20220526') & (x <= '20220712'):
    return 1.75
  elif (x >= '20220414') & (x <= '20220525'):
    return 1.50
  elif (x >= '20220114') & (x <= '20220413'):
    return 1.25
  elif (x >= '20211125') & (x <= '20220113'):
    return 1.00	
  elif (x >= '20210826') & (x <= '20211124'):
    return 0.75
  elif (x >= '20200528') & (x <= '20210825'):
    return 0.50	
  elif (x >= '20200317') & (x <= '20200316'):
    return 0.75
  elif (x >= '20190718') & (x <= '20191015'):
    return 1.25
  elif (x >= '20181130') & (x <= '20190717'):
    return 1.75
  elif (x >= '20171130') & (x <= '20181129'):
    return 1.50
  elif (x >= '20160609') & (x <= '20171129'):
    return 1.25
  elif (x >= '20150611') & (x <= '20160608'):
    return 1.50
  elif (x >= '20150312') & (x <= '20150610'):
    return 1.75	
  elif (x >= '20141015') & (x <= '20150311'):
    return 2.00
  elif (x >= '20140814') & (x <= '20141014'):
    return 2.25	
  elif (x >= '20130509') & (x <= '20140813'):
    return 2.50		
  else:
    return 2.75	


df['baserate']=''
df['baserate'] = df.date.apply(classi_rate)

In [95]:
df.to_csv('Apt_total_eda.csv', encoding='utf-8')

##### **데이터전처리 2**

In [124]:
filename = 'Apt_total_eda.csv'
df = pd.read_csv(filename, encoding='utf-8', encoding_errors='ignore', engine='python', index_col=0).reset_index(drop=True) #30초
df = df.astype({'date_year' : 'str', 'date_month' : 'str'})
df.head()

,date,address,date_year,date_month,date_day,add_gu,add_dong,address_num,address_num_first,address_num_second,...,price,date_on_sale,deal_type,real_estate_office,cpi,area_type,building_year,river,apt_brand,baserate
0,20130101,서울특별시 은평구 대조동,2013,1,1,은평구,대조동,04월 18일,4.0,18.0,...,16900,NaN,-,-,93.134,1,1.0,0,0.0,2.75
1,20130101,서울특별시 구로구 구로동,2013,1,1,구로구,구로동,1280,1280.0,0.0,...,34500,NaN,-,-,93.134,2,6.0,0,0.0,2.75
2,20130101,서울특별시 동작구 사당동,2013,1,1,동작구,사당동,1155,1155.0,0.0,...,40250,NaN,-,-,93.134,3,4.0,0,0.0,2.75
3,20130101,서울특별시 은평구 응암동,2013,1,1,은평구,응암동,176,176.0,0.0,...,27500,NaN,-,-,93.134,2,5.0,0,2.0,2.75
4,20130101,서울특별시 중구 신당동,2013,1,1,중구,신당동,851,851.0,0.0,...,62000,NaN,-,-,93.134,4,2.0,0,2.0,2.75


In [109]:
df_park[df_park.year == '2013']

,add_gu,count,duplicate,area,year,area_gu,ratio_park
209,소계,"2,782",34,170080900.0,2013,NaN,NaN
210,종로구,114,2,11575400.0,2013,23910000.0,0.484124
211,중구,68,2,3128100.0,2013,9960000.0,0.314066
212,용산구,101,1,1796000.0,2013,21870000.0,0.082122
213,성동구,80,1,3061700.0,2013,16860000.0,0.181595
214,광진구,55,2,3395600.0,2013,17060000.0,0.199039
215,동대문구,102,1,1234500.0,2013,14220000.0,0.086814
216,중랑구,103,-,5230700.0,2013,18500000.0,0.282741
217,성북구,130,5,8391200.0,2013,24580000.0,0.341383
218,강북구,83,1,14336600.0,2013,23600000.0,0.607483


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784225 entries, 0 to 784224
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   date                784225 non-null  int64  
 1   address             784225 non-null  object 
 2   date_year           784225 non-null  int64  
 3   date_month          784225 non-null  int64  
 4   date_day            784225 non-null  int64  
 5   add_gu              784225 non-null  object 
 6   add_dong            784225 non-null  object 
 7   address_num         784004 non-null  object 
 8   address_num_first   784150 non-null  float64
 9   address_num_second  784150 non-null  float64
 10  address_street      784225 non-null  object 
 11  apt_name            784225 non-null  object 
 12  area                784225 non-null  float64
 13  floor               784224 non-null  float64
 14  construction_year   784223 non-null  float64
 15  price               784225 non-nul

In [133]:
# park 정보
list(df_park.add_gu.unique())

lst_gu =  ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
 '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구',
 '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']


# 연도별로 

def classi_park(x):
  y = df[df['date_year'] == x] #df에서 해당하는 연도만 추출한 데이터프레임

  df2 = df.copy()
  for i in range(0, len(lst_gu)): #30초!!!
    result = y[y['add_gu'].str.contains(lst_gu[i])] #연도 + 해당하는 구의 인덱스만 추출
    df2.loc[list(result.index), 'park_area'] = df_park[df_park.year == x].iloc[i+1, 3] #그인덱스에 df_park의 값 집어넣기
    df2.loc[list(result.index), 'park_count'] = df_park[df_park.year == x].iloc[i+1, 1]
    df2.loc[list(result.index), 'park_ratio'] = df_park[df_park.year == x].iloc[i+1, 6]


  df.loc[list(df2.index), 'park_area'] = df2.loc[list(df2.index), 'park_area']
  df.loc[list(df2.index), 'park_count'] = df2.loc[list(df2.index), 'park_count']
  df.loc[list(df2.index), 'park_ratio'] = df2.loc[list(df2.index), 'park_ratio']

classi_park('2013')
classi_park('2014')
classi_park('2015')
classi_park('2016')
classi_park('2017')
classi_park('2018')
classi_park('2019')
classi_park('2020')
classi_park('2021') #30초~2분


In [135]:
y = df[df['date_year'] == '2022'] #df에서 해당하는 연도만 추출한 데이터프레임

df2 = df.copy()
for i in range(0, len(lst_gu)): #30초!!!
  result = y[y['add_gu'].str.contains(lst_gu[i])] #연도 + 해당하는 구의 인덱스만 추출
  df2.loc[list(result.index), 'park_area'] = df_park[df_park.year == '2021'].iloc[i+1, 3] #그인덱스에 df_park의 값 집어넣기
  df2.loc[list(result.index), 'park_count'] = df_park[df_park.year == '2021'].iloc[i+1, 1]
  df2.loc[list(result.index), 'park_ratio'] = df_park[df_park.year == '2021'].iloc[i+1, 6]


df.loc[list(df2.index), 'park_area'] = df2.loc[list(df2.index), 'park_area']
df.loc[list(df2.index), 'park_count'] = df2.loc[list(df2.index), 'park_count']
df.loc[list(df2.index), 'park_ratio'] = df2.loc[list(df2.index), 'park_ratio']

In [136]:
# EDA 확인
df.iloc[780500]

date                       20220531
address               서울특별시 노원구 공릉동
date_year                      2022
date_month                        5
date_day                         31
add_gu                          노원구
add_dong                        공릉동
address_num                     715
address_num_first             715.0
address_num_second              0.0
address_street              섬밭로 123
apt_name               시영3차(라이프)아파트
area                          34.44
floor                          15.0
construction_year            1994.0
price                         42000
date_on_sale                    NaN
deal_type                      중개거래
real_estate_office           서울 노원구
cpi                          107.56
area_type                         2
building_year                  28.0
river                             0
apt_brand                       0.0
baserate                       1.75
park_area                14742900.0
park_count                      190
park_ratio                 0

In [ ]:
df.head()

,date,address,date_year,date_month,date_day,add_gu,add_dong,address_num,address_num_first,address_num_second,...,floor,construction_year,price,date_on_sale,deal_type,real_estate_office,building_year,river,baserate,apt_brand
0,20130101,서울특별시 은평구 대조동,2013,1,1,은평구,대조동,04월 18일,4.0,18.0,...,9.0,2012.0,16900,NaN,-,-,1.0,0,2.75,0.0
1,20130101,서울특별시 구로구 구로동,2013,1,1,구로구,구로동,1280,1280.0,0.0,...,12.0,2007.0,34500,NaN,-,-,6.0,0,2.75,0.0
2,20130101,서울특별시 동작구 사당동,2013,1,1,동작구,사당동,1155,1155.0,0.0,...,7.0,2009.0,40250,NaN,-,-,4.0,0,2.75,0.0
3,20130101,서울특별시 은평구 응암동,2013,1,1,은평구,응암동,176,176.0,0.0,...,9.0,2008.0,27500,NaN,-,-,5.0,0,2.75,2.0
4,20130101,서울특별시 중구 신당동,2013,1,1,중구,신당동,851,851.0,0.0,...,14.0,2011.0,62000,NaN,-,-,2.0,0,2.75,2.0


#### **시각화**

In [ ]:
기준금리에 따른 가격분포
면적에 따른 가격분포
숲면적에 따른 가격분포

In [ ]:
# 전역설정 / 한글폰트
plt.rc('font', family='NanumBarunGothic') 

#### **데이터 스플릿**

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784225 entries, 0 to 784224
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   date                784225 non-null  int64  
 1   address             784225 non-null  object 
 2   date_year           784225 non-null  object 
 3   date_month          784225 non-null  object 
 4   date_day            784225 non-null  int64  
 5   add_gu              784225 non-null  object 
 6   add_dong            784225 non-null  object 
 7   address_num         784004 non-null  object 
 8   address_num_first   784150 non-null  float64
 9   address_num_second  784150 non-null  float64
 10  address_street      784225 non-null  object 
 11  apt_name            784225 non-null  object 
 12  area                784225 non-null  float64
 13  floor               784224 non-null  float64
 14  construction_year   784223 non-null  float64
 15  price               784225 non-nul

In [148]:
df = df.astype({'date_year' : 'int', 'date_month' : 'int', 'park_count':'int'})
train = df[df['date_year'] <= 2018]
val = df[(df['date_year'] <= 2020) & (df['date_year'] >= 2019) ]
test = df[df['date_year'] >= 2021]

#### **모델 학습**

In [143]:
#단순선형회귀모델
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 데이터스플릿

target = 'price'
feature = 'area'

X_train = train[feature] 
y_train = train[target]
X_val = val[feature]  
y_val = val[target]


#모델학습
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder(cols=[], mapping=[])),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

#### **모델 평가**

In [144]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델 정확도 계산 / .score()
# R2(결정계수)는 예측값 도출하지 않아도(.predict) 바로 계산 가능
score = pipe.score(X_train, y_train) 
print(f'훈련 에러(R2): {score:.2f}') 
score = pipe.score(X_val, y_val) 
print(f'테스트 에러(R2): {score:.2f}')


# 지표계산 및 출력 / 인자
y_pred_train = pipe.predict(X_train) 
y_pred_val = pipe.predict(X_val)

mae_train = mean_absolute_error(y_train, y_pred_train)  #훈련셋의 MAE
mae_test = mean_absolute_error(y_val, y_pred_val) #테스트셋의 MAE

print(f'훈련 에러: {mae_train:.2f}') 
print(f'검증 에러: {mae_test:.2f}')


# 지표계산 및 출력 / 데이터프레임
mse = mean_squared_error(y_val, y_pred_val)
mae = mean_absolute_error(y_val, y_pred_val)
rmse = mse ** 0.5
r2 = r2_score(y_val, y_pred_val)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1
rmse = mse ** 0.5
r2 = r2_score(y_val, y_pred_val)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1

# 단순선형회귀모델
# 훈련 에러(R2): 0.39
# 테스트 에러(R2): -0.00
# 훈련 에러: 20695.60
# 테스트 에러: 43263.18

# Metric	Score
# 0	MSE	4.556139e+09
# 1	MAE	4.326318e+04
# 2	RMSE	6.749917e+04
# 3	R2	-0.000000e+00


# 다중선형회귀모델
# 시계열로 나눠서 그런지 훈련에러와 테스트에러 차이가 너무 크게 나오네... 다른 모델로 구현해도 그럴까?
# 훈련 에러(R2): 0.45 
# 테스트 에러(R2): 0.26
# 훈련 에러: 19769.78

# 테스트 에러: 34129.16
# Metric	Score
# 0	MSE	3.355991e+09
# 1	MAE	3.412916e+04
# 2	RMSE	5.793092e+04
# 3	R2	2.600000e-01

훈련 에러(R2): 0.41
테스트 에러(R2): 0.04
훈련 에러: 18969.63
검증 에러: 36081.80


,Metric,Score
0,MSE,3.134206e+09
1,MAE,3.608180e+04
2,RMSE,5.598398e+04
3,R2,4.000000e-02


In [152]:
#다중선형회귀모델
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 데이터스플릿

target = 'price'
feature = ['date_year', 'date_month', 'add_gu',
          'add_dong','apt_name', 'area', 'floor','deal_type', 'cpi',
          'area_type', 'building_year', 'river', 'apt_brand', 'baserate',
          'park_area', 'park_count', 'park_ratio']

X_train = train[feature] 
y_train = train[target]
X_val = val[feature]  
y_val = val[target]


#모델학습
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

pipe.fit(X_train, y_train) 

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['add_gu', 'add_dong', 'apt_name',
                                      'deal_type'],
                                mapping=[{'col': 'add_gu',
                                          'data_type': dtype('O'),
                                          'mapping': 은평구      1
구로구      2
동작구      3
중구       4
관악구      5
동대문구     6
강서구      7
금천구      8
양천구      9
서초구     10
마포구     11
성북구     12
강남구     13
노원구     14
도봉구     15
중랑구     16
서대문구    17
종로구     18
송파구     19
강동구     20
용산구     21
강북구     22
성동구     23
영등포구    24
광진구     25
NaN     -2
dtype: int64},
                                         {'col': 'add_dong',
                                          'data_type': dtype('O'),
                                          'mapping': 대조동      1
구로...
                                         {'col': 'apt_name',
                                          'data_type': dtype('O'),
                                   

In [153]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델 정확도 계산 / .score()
# R2(결정계수)는 예측값 도출하지 않아도(.predict) 바로 계산 가능
score = pipe.score(X_train, y_train) 
print(f'훈련 에러(R2): {score:.2f}') 
score = pipe.score(X_val, y_val) 
print(f'테스트 에러(R2): {score:.2f}')


# 지표계산 및 출력 / 인자
y_pred_train = pipe.predict(X_train) 
y_pred_val = pipe.predict(X_val)

mae_train = mean_absolute_error(y_train, y_pred_train)  #훈련셋의 MAE
mae_test = mean_absolute_error(y_val, y_pred_val) #테스트셋의 MAE

print(f'훈련 에러: {mae_train:.2f}') 
print(f'검증 에러: {mae_test:.2f}')


# 지표계산 및 출력 / 데이터프레임
mse = mean_squared_error(y_val, y_pred_val)
mae = mean_absolute_error(y_val, y_pred_val)
rmse = mse ** 0.5
r2 = r2_score(y_val, y_pred_val)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1
rmse = mse ** 0.5
r2 = r2_score(y_val, y_pred_val)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1

훈련 에러(R2): 0.59
테스트 에러(R2): 0.45
훈련 에러: 16041.79
검증 에러: 27877.70


,Metric,Score
0,MSE,1.798332e+09
1,MAE,2.787770e+04
2,RMSE,4.240674e+04
3,R2,4.500000e-01
